In [1]:
import requests
s = requests.Session()
s.headers = {
    #"referer" : "http://www.google.com",
    "user-agent" : "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
}
from bs4 import BeautifulSoup
import re

In [2]:
""" Form Google search urls for scraping """

GOOGLE_ORIG_URL = 'https://www.google.com/search?q=carbon+footprint'

page_amount = 11
GOOGLE_URLS = [GOOGLE_ORIG_URL + '&start=' + str(i*10) for i in range(0, page_amount)]
print(GOOGLE_URLS)

""" Scrape Google search results """

def get_response(url):
    resp = s.get(url)
    if resp.status_code != 200:
        print(resp.text)
        raise
    return resp

google_responses = [get_response(url) for url in GOOGLE_URLS]
print(google_responses)

""" Parse scraped Google search results """

def google_get_description(response):
    returnable=[]
    soup = BeautifulSoup(response.content, "html.parser")
    for s in soup.find_all("div", class_="g"):
        spans = s.find_all('span', class_=None)
        without_attrs = [s for s in spans if (s.attrs == {})]
        without_feedback = [s for s in without_attrs if 'Feedback' != s.text]
        without_cached = [s for s in without_feedback if 'Cached' != s.text]
        without_similar = [s for s in without_cached if 'Similar' != s.text]
        without_cachedsimilar = [s for s in without_similar if 'CachedSimilar' != s.text]
        for desc in without_cachedsimilar:
            desc_text = desc.text
            returnable.append(desc_text.replace('\xa0', ' '))
    return returnable

google_descriptions_of_pages = [google_get_description(res) for res in google_responses]

def flatten(t):
    return [item for sublist in t for item in sublist]

bs4_scraped_snippets = flatten(google_descriptions_of_pages)

print(*bs4_scraped_snippets[:15], sep='\n')
print(len(bs4_scraped_snippets))
# Possibly not usable data atm
# Although it doesn't cost anything to calc the metrics for this data also

with open("google_requests_scrape.csv", "w") as s_file:
    s_file.write("\n".join(bs4_scraped_snippets))

['https://www.google.com/search?q=carbon+footprint&start=0', 'https://www.google.com/search?q=carbon+footprint&start=10', 'https://www.google.com/search?q=carbon+footprint&start=20', 'https://www.google.com/search?q=carbon+footprint&start=30', 'https://www.google.com/search?q=carbon+footprint&start=40', 'https://www.google.com/search?q=carbon+footprint&start=50', 'https://www.google.com/search?q=carbon+footprint&start=60', 'https://www.google.com/search?q=carbon+footprint&start=70', 'https://www.google.com/search?q=carbon+footprint&start=80', 'https://www.google.com/search?q=carbon+footprint&start=90', 'https://www.google.com/search?q=carbon+footprint&start=100']
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
A carbon footprint is the total amount of greenhouse gases (including carbon dioxide and methane) that are generated by our actio

In [3]:
""" BBC scrape """

BBC_ORIG_URL = 'https://www.bbc.co.uk/search?q=carbon+footprint'

page_amount = 11
BBC_URLS = [BBC_ORIG_URL + '&page=' + str(i) for i in range(1, page_amount)]
#print(BBC_URLS)

bbc_responses = [get_response(url) for url in BBC_URLS]

def get_bbc_heading(response):
    returnable = []
    soup = BeautifulSoup(response.content, "html.parser")
    regex = re.compile('.*-Promo.')
    for a in soup.find_all("a", {"class" : regex}):
        returnable.append(a.text)
    return returnable

def get_bbc_description(response):
    returnable = []
    soup = BeautifulSoup(response.content, "html.parser")
    regex = re.compile('.*-Paragraph.')
    for p in soup.find_all("p", {"class" : regex}):
        if '© 2021 BBC. The BBC is not responsible for the content of external sites. Read about our approach to external linking.' in p.text:
            continue
        returnable.append(p.text)
    return returnable

bbc_headings_of_pages = [get_bbc_heading(res) for res in bbc_responses]
bbc_scraped_headings = flatten(bbc_headings_of_pages)
print(*bbc_scraped_headings[:15], sep='\n')
print(len(bbc_scraped_headings))

with open("bbc_requests_headings_scrape.csv", "w") as s_file:
    s_file.write("\n".join(bbc_scraped_headings))

print('\n-----\n')

bbc_descriptions_of_pages = [get_bbc_description(res) for res in bbc_responses]
bbc_scraped_snippets = flatten(bbc_descriptions_of_pages)
print(*bbc_scraped_snippets[:15], sep='\n')
print(len(bbc_scraped_snippets))

with open("bbc_requests_snippets_scrape.csv", "w") as s_file:
    s_file.write("\n".join(bbc_scraped_snippets))

Dani's House: Series 1: It's Not Easy Being Green
Horizon: 2021: Feast to Save the Planet
Shop Well for the Planet?: Series 1: Episode 2
Clacton Airshow looks to reduce carbon footprint
Landward: 2021: Episode 18
Countryfile: Offa's Dyke Path
Royal Institution Christmas Lectures: 2020: Planet Earth - A User's Guide: Up in the Air
Hayley Goes...: Series 2: Back to Nature
Shop Well for the Planet?: Series 1: Episode 1
Newcastle's plastic grass sparks 'carbon footprint' row
Climate change: Could Welsh farms help meet targets?
Firms want help to measure their carbon footprint
Do consumers care about carbon footprint?
COP26: Climate change what do all the words mean?
Farming Today: 19/08/21 - The carbon footprint of milk, Wheat harvest in Yorkshire
100

-----

Sam dares Dani and Toby to cut their carbon footprint.
A dinner party where special guests are scored on the environmental impact of their meal.
Can the gang get a whole family on board this eco and money-saving mission?
The Clacton A

In [4]:
""" The Register scrape """

The_Register_ORIG_URL = 'https://search.theregister.com/?q=carbon+footprint&results_per_page=10&sort=rel'

page_amount = 11
The_Register_URLS = [The_Register_ORIG_URL + '&page=' + str(i) for i in range(1, page_amount)]

the_register_responses = [get_response(url) for url in The_Register_URLS]

def get_the_register_heading(response):
    returnable = []
    soup = BeautifulSoup(response.content, "html.parser")
    for article in soup.find_all("article"):
        for h4 in article.find_all("h4"):
            returnable.append(h4.text)
    return returnable

def get_the_register_description(response):
    returnable = []
    soup = BeautifulSoup(response.content, "html.parser")
    for article in soup.find_all("article"):
        for div in article.find_all("div", class_="standfirst"):
            returnable.append(div.text)
    return returnable

the_register_headings_of_pages = [get_the_register_heading(res) for res in the_register_responses]
the_register_scraped_headings = flatten(the_register_headings_of_pages)
print(*the_register_scraped_headings[:15], sep='\n')
print(len(the_register_scraped_headings))

with open("register_requests_headings_scrape.csv", "w") as s_file:
    s_file.write("\n".join(the_register_scraped_headings))

print('\n-----\n')

the_register_descriptions_of_pages = [get_the_register_description(res) for res in the_register_responses]
the_register_scraped_snippets = flatten(the_register_descriptions_of_pages)
print(*the_register_scraped_snippets[:15], sep='\n')
print(len(the_register_scraped_snippets))

with open("register_requests_snippets_scrape.csv", "w") as s_file:
    s_file.write("\n".join(the_register_scraped_snippets))

Amazon: Our carbon footprint went up 19% last year but we grew even more than that, so 'carbon intensity' is down
Measuring your carbon footprint? There's no app for that
AI me to the Moon... Carbon footprint for 'training GPT-3' same as driving to our natural satellite and back
Microsoft picks a side, aims to make the business 'carbon-negative' by 2030
Azure Emissions Dashboard shows how you and Microsoft are slowly killing the planet with your cloud workloads
Google Cloud will let you know how your workloads are damaging the environment
Japan's NTT Group to allow remote work for all 320,000 staff
Altered carbon: Boffins automate DNA storage with decent density – but lousy latency
Wipro wins $44.5m deal for data centres and managed services at UK's National Grid
'Green' Apple: We've smudged a bit off our carbon footprint
Brit MPs to Apple CEO: Please stop ignoring our questions about repairability and the environment
Bill Gates on climate change: Planting trees is not the answer, emis

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df8df42c-8503-4bb3-8c9d-497cf36bf82c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>